# RAG Evaluation Retrieval

In [2]:
import minsearch
import json
import pandas as pd


from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from sklearn.model_selection import train_test_split

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

## Ingestion

In [3]:
with open('notebooks/documents-with-ids.json', 'r') as file:
    documents = json.load(file)

## Index and mapping Minsearch

In [4]:
index = minsearch.Index(
    text_fields=["Question", "Answer", "Category"],
    keyword_fields=["Section", "id"]
)

In [5]:
index.fit(documents)

def min_search(query, section):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=5
    )

    return results

## RAG flow

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You are an expert in United Kingdom Benefit Claims and Medical Negligence Claims. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
query = "Can I get sick pay if I'm self-isolating?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

# Evaluating Retrieval
- hit_rate
- Mean Reciprocal Rank

## Minsearch without tuning

In [9]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [10]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [11]:
df_ground_truth = pd.read_csv('notebooks/ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

evaluate(ground_truth, lambda q: min_search(q['question'], q['claims_type']))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:04<00:00, 427.99it/s]


{'hit_rate': 0.7965936739659367, 'mrr': 0.6443309002433094}

## Hyperparameter tuning

In [13]:
space = {
    'num_results': scope.int(hp.quniform('num_results', 1, 10, 1)),
    'boost_factor': hp.uniform('boost_factor', 0.1, 2.0),
    'section': hp.choice('section', ['nhs claim benefits', 'general claim benefits'])
}

In [14]:
def objective(params):
    num_results = params['num_results']
    boost_factor = params['boost_factor']
    section = params['section']
    
    def tuned_min_search(query, section):
        boost = {
            "Question": boost_factor,
            "Answer": boost_factor
        }

        filter_dict = {
            'section': section  # Pass the optimized section value
        }
        
        results = index.search(
            query=query,
            filter_dict=filter_dict,
            boost_dict=boost,
            num_results=num_results
        )
        return results

    metrics = evaluate(ground_truth, lambda q: tuned_min_search(q['question'], q['claims_type']))
    return {'loss': -metrics['hit_rate'], 'status': STATUS_OK}

In [15]:
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

  0%|                                                                                                  | 0/50 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/2055 [00:00<?, ?it/s]

  2%|1         | 37/2055 [00:00<00:05, 362.43it/s]

  4%|3         | 81/2055 [00:00<00:04, 407.01it/s]

  6%|6         | 127/2055 [00:00<00:04, 431.07it/s]

  8%|8         | 174/2055 [00:00<00:04, 445.74it/s]

 11%|#         | 220/2055 [00:00<00:04, 450.30it/s]

 13%|#2        | 266/2055 [00:00<00:04, 428.26it/s]

 15%|#5        | 310/2055 [00:00<00:05, 343.88it/s]

 17%|#6        | 347/2055 [00:00<00:04, 342.69it/s]

 19%|#8        | 383/2055 [00:01<00:04, 345.91it/s]

 21%|##        | 430/2055 [00:01<00:04, 378.28it/s]

 23%|##2       | 469/2055 [00:01<00:04, 356.91it/s]

 25%|##4       | 509/2055 [00:01<00:04, 367.25it/s]

 27%|##7       | 555/2055 [00:01<00:03, 392.68it/s]

 29%|##9       | 601/2055 [00:01<00:03, 410.52it/s]

 31%|###1      | 646/2055 [00:01<00:03, 421.41it/s]

 34%|###3      | 692/2055 [00:01<00:03, 431.40it/s]

 36%|###5      | 739/2055 [00:01<00:02, 440.47it/s]

 38%|###8      | 784/2055 [00:01<00:02, 436.82it/s]

 40%|#

  2%|█▍                                                                     | 1/50 [00:04<03:57,  4.85s/trial, best loss: -0.6165450121654501]

  0%|          | 0/2055 [00:00<?, ?it/s]

  2%|2         | 44/2055 [00:00<00:04, 435.88it/s]

  4%|4         | 91/2055 [00:00<00:04, 450.92it/s]

  7%|6         | 137/2055 [00:00<00:04, 454.70it/s]

  9%|8         | 183/2055 [00:00<00:04, 453.10it/s]

 11%|#1        | 230/2055 [00:00<00:04, 455.82it/s]

 13%|#3        | 276/2055 [00:00<00:03, 454.79it/s]

 16%|#5        | 322/2055 [00:00<00:03, 450.66it/s]

 18%|#7        | 368/2055 [00:00<00:03, 452.33it/s]

 20%|##        | 415/2055 [00:00<00:03, 454.80it/s]

 22%|##2       | 461/2055 [00:01<00:03, 455.73it/s]

 25%|##4       | 507/2055 [00:01<00:03, 447.25it/s]

 27%|##6       | 554/2055 [00:01<00:03, 452.15it/s]

 29%|##9       | 600/2055 [00:01<00:03, 454.26it/s]

 31%|###1      | 646/2055 [00:01<00:03, 376.70it/s]

 33%|###3      | 686/2055 [00:01<00:03, 381.01it/s]

 35%|###5      | 726/2055 [00:01<00:03, 383.91it/s]

 38%|###7      | 771/2055 [00:01<00:03, 401.16it/s]

 40%|###9      | 813/2055 [00:01<00:03, 379.25it/s]

 42%|#

  4%|██▉                                                                     | 2/50 [00:09<03:49,  4.79s/trial, best loss: -0.683698296836983]

...

 96%|███████████████████████████████████████████████████████████████████▏  | 48/50 [03:52<00:09,  4.87s/trial, best loss: -0.9060827250608272]

  0%|          | 0/2055 [00:00<?, ?it/s]

  2%|2         | 47/2055 [00:00<00:04, 460.81it/s]

  5%|4         | 94/2055 [00:00<00:04, 450.31it/s]

  7%|6         | 140/2055 [00:00<00:04, 453.54it/s]

  9%|9         | 186/2055 [00:00<00:04, 453.67it/s]

 11%|#1        | 232/2055 [00:00<00:04, 454.29it/s]

 14%|#3        | 278/2055 [00:00<00:03, 445.47it/s]

 16%|#5        | 324/2055 [00:00<00:03, 449.07it/s]

 18%|#8        | 370/2055 [00:00<00:03, 452.08it/s]

 20%|##        | 416/2055 [00:00<00:03, 452.62it/s]

 22%|##2       | 462/2055 [00:01<00:03, 452.07it/s]

 25%|##4       | 508/2055 [00:01<00:03, 452.17it/s]

 27%|##6       | 554/2055 [00:01<00:03, 410.96it/s]

 29%|##9       | 596/2055 [00:01<00:03, 390.24it/s]

 31%|###1      | 642/2055 [00:01<00:03, 408.40it/s]

 33%|###3      | 686/2055 [00:01<00:03, 413.59it/s]

 35%|###5      | 728/2055 [00:01<00:03, 400.34it/s]

 38%|###7      | 774/2055 [00:01<00:03, 416.28it/s]

 40%|###9      | 820/2055 [00:01<00:02, 426.38it/s]

 42%|#

 98%|████████████████████████████████████████████████████████████████████▌ | 49/50 [03:57<00:04,  4.88s/trial, best loss: -0.9094890510948905]

  0%|          | 0/2055 [00:00<?, ?it/s]

  2%|2         | 44/2055 [00:00<00:04, 436.45it/s]

  4%|4         | 88/2055 [00:00<00:05, 381.92it/s]

  6%|6         | 130/2055 [00:00<00:04, 393.50it/s]

  8%|8         | 170/2055 [00:00<00:05, 363.53it/s]

 11%|#         | 216/2055 [00:00<00:04, 393.28it/s]

 13%|#2        | 262/2055 [00:00<00:04, 412.52it/s]

 15%|#4        | 305/2055 [00:00<00:04, 416.59it/s]

 17%|#6        | 347/2055 [00:00<00:04, 407.39it/s]

 19%|#8        | 390/2055 [00:00<00:04, 411.72it/s]

 21%|##1       | 436/2055 [00:01<00:03, 425.37it/s]

 23%|##3       | 479/2055 [00:01<00:03, 421.84it/s]

 26%|##5       | 525/2055 [00:01<00:03, 430.73it/s]

 28%|##7       | 571/2055 [00:01<00:03, 437.04it/s]

 30%|##9       | 615/2055 [00:01<00:03, 409.59it/s]

 32%|###1      | 657/2055 [00:01<00:03, 398.11it/s]

 34%|###4      | 703/2055 [00:01<00:03, 413.23it/s]

 36%|###6      | 746/2055 [00:01<00:03, 417.19it/s]

 39%|###8      | 792/2055 [00:01<00:02, 427.85it/s]

 41%|#

100%|██████████████████████████████████████████████████████████████████████| 50/50 [04:02<00:00,  4.85s/trial, best loss: -0.9094890510948905]


In [16]:
df_full_train, df_test = train_test_split(ground_truth, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [17]:
best_num_results = int(best['num_results'])
best_boost_factor = best['boost_factor']
best_section_index = best['section']


section_options = ['nhs claim benefits', 'general claim benefits']
 
best_filter_dict = {'section': section_options[best_section_index]}

In [18]:
def best_min_search(query, section):
    boost = {
        "Question": best_boost_factor,
        "Answer": best_boost_factor
    }
    
    results = index.search(
        query=query,
        filter_dict=best_filter_dict,
        boost_dict=boost,
        num_results=best_num_results
    )
    return results

# Evaluate using the best parameters
metrics = evaluate(df_val, lambda q: best_min_search(q['question'], q['claims_type']))
print(metrics)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 411/411 [00:01<00:00, 402.91it/s]

{'hit_rate': 0.9051094890510949, 'mrr': 0.712078554049357}


In [19]:
print(best_num_results)
print(best_boost_factor)
print(best_filter_dict)

10
1.9376603614188619
{'section': 'general claim benefits'}


In [20]:
def min_search(query, section):
    boost = {
        "Question": 1.76740328091659, 
        "Answer": 1.76740328091659     
    }

    results = index.search(
        query=query,
        filter_dict={'section': section},  
        boost_dict=boost,                  
        num_results=10                     
    )

    return results

In [21]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']  # Ground truth document ID
        results = search_function(q['question'], q['claims_type'])  # Pass query and section
        relevance = [d['id'] == doc_id for d in results]  # Check if doc ID matches in results
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# Evaluate on the test dataset
metrics = evaluate(df_test, lambda question, claims_type: min_search(question, claims_type))

print(metrics)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 411/411 [00:00<00:00, 424.67it/s]

{'hit_rate': 0.8880778588807786, 'mrr': 0.7030423280423281}


In [ ]:
Minsearch metrics without tuning {'hit_rate': 0.7965936739659367, 'mrr': 0.6443309002433094}

In [46]:
import json


from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

In [2]:
with open('notebooks/documents-with-ids.json', 'r') as file:
    documents = json.load(file)

documents[10]

{'Category': 'Temporarily unable to work',
 'Question': "Can I get sick pay if I'm self-isolating?",
 'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
 'Section': 'general claim benefits',
 'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'}

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Answer": {"type": "text"},
            "Category": {"type": "text"},
            "Question": {"type": "text"},
            "Section": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'benefit-claims'})

In [4]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/425 [00:00<?, ?it/s]

In [5]:
def elastic_search(query, section):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Question^3", "Answer", "Category"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "Section": section
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [6]:
elastic_search(
    query="Can I get sick pay if I'm self-isolating?",
    section="general claim benefits"
)

[{'Category': 'Temporarily unable to work',
  'Question': "Can I get sick pay if I'm self-isolating?",
  'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
  'Section': 'general claim benefits',
  'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'},
 {'Category': 'Temporarily unable to work',
  'Question': 'How do I apply for sick pay?',
  'Answer': 'You need to provide a fit note from your doctor to apply for sick pay.',
  'Section': 'general claim benefits',
  'id': 'ecc42084-d88e-5bf0-8070-a18552c283bb'},
 {'Category': 'Temporarily unable to work',
  'Question': 'What is statutory sick pay?',
  'Answer': "Statutory Sick Pay is a legal requirement for employers to pay you if you're ill.",
  'Section': 'general claim benefits',
  'id': 'ee6a77ec-d4ef-50f7-9762-04cc228b3a48'},
 {'Category': 'Disabled or health condition',
  'Question': 'Can I get help with housing if I’m disabled?',
  'Answer': 'You may be eligible for a Disabled Facilities Grant to adapt your home to

In [47]:
import pandas as pd

In [7]:
df_ground_truth = pd.read_csv('notebooks/ground-truth-data.csv')

In [8]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [9]:
ground_truth[10]

{'question': 'Is it possible to appeal?',
 'claims_type': 'general claim benefits',
 'document': '8d000ade-6c2b-571c-aa61-5d38eb463cf8'}

In [10]:
elastic_search(
    query="Can I get sick pay if I'm self-isolating?",
    section="general claim benefits"
)

[{'Category': 'Temporarily unable to work',
  'Question': "Can I get sick pay if I'm self-isolating?",
  'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
  'Section': 'general claim benefits',
  'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'},
 {'Category': 'Temporarily unable to work',
  'Question': 'How do I apply for sick pay?',
  'Answer': 'You need to provide a fit note from your doctor to apply for sick pay.',
  'Section': 'general claim benefits',
  'id': 'ecc42084-d88e-5bf0-8070-a18552c283bb'},
 {'Category': 'Temporarily unable to work',
  'Question': 'What is statutory sick pay?',
  'Answer': "Statutory Sick Pay is a legal requirement for employers to pay you if you're ill.",
  'Section': 'general claim benefits',
  'id': 'ee6a77ec-d4ef-50f7-9762-04cc228b3a48'},
 {'Category': 'Disabled or health condition',
  'Question': 'Can I get help with housing if I’m disabled?',
  'Answer': 'You may be eligible for a Disabled Facilities Grant to adapt your home to

In [11]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], section=q['claims_type'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/2055 [00:00<?, ?it/s]

In [12]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [13]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [14]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7732360097323601, 0.6168369829683708)

In [15]:
documents[10]

{'Category': 'Temporarily unable to work',
 'Question': "Can I get sick pay if I'm self-isolating?",
 'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
 'Section': 'general claim benefits',
 'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'}

## Minsearch

In [16]:
import minsearch

index = minsearch.Index(
    text_fields=["Question", "Answer", "Category"],
    keyword_fields=["Section", "id"]
)

index.fit(documents)

In [37]:
def min_search(query, section):
    boost = {'Question': 3.0, 'Category': 0.5}

    results = index.search(
        query=query,
        filter_dict={'Section': section},
        boost_dict=boost,
        num_results=5
    )

    return results

In [38]:
ground_truth[0]

{'question': 'How can I change my existing benefit details?',
 'claims_type': 'general claim benefits',
 'document': '30eada08-5708-5c5c-9df8-0f7d5d4dc131'}

In [39]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = min_search(query=q['question'], section=q['claims_type'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/2055 [00:00<?, ?it/s]

In [40]:
hit_rate(relevance_total), mrr(relevance_total)

(0.8306569343065694, 0.6697891321978915)

`ES- Hitrate:0.7732360097323601, MRR:0.6168369829683708)`

In [41]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [43]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['claims_type']))

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.7732360097323601, 'mrr': 0.616999188969993}

In [45]:
evaluate(ground_truth, lambda q: min_search(q['question'], q['claims_type']))

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.8306569343065694, 'mrr': 0.6697891321978915}

## Vector Search

In [48]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/home/codespace/.local/share/virtualenvs/uk-benefits-assistant-nL9c5Gpa/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [49]:
v = model.encode("Can I get sick pay if I'm self-isolating?")

In [50]:
len(v)

384

In [85]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Answer": {"type": "text"},
            "category": {"type": "text"},
            "Question": {"type": "text"},
            "Section": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'benefit-claims'})

In [86]:
documents[0]

{'Category': 'Manage existing benefit',
 'Question': 'How do I update my benefit information?',
 'Answer': 'You can update your benefit information online through your account.',
 'Section': 'general claim benefits',
 'id': '30eada08-5708-5c5c-9df8-0f7d5d4dc131',
 'question_vector': array([-1.80060193e-02,  5.96722253e-02,  1.26943989e-02, -3.90117168e-02,
         2.84865871e-02,  9.22173411e-02,  2.27035414e-02,  2.67567299e-02,
        -3.86952274e-02, -2.83248979e-03, -2.74528917e-02,  2.94101797e-03,
        -3.46847787e-03, -1.18925475e-01,  1.39166079e-02,  3.98957506e-02,
        -1.79248862e-02,  7.15748966e-02, -4.80940826e-02, -3.22361998e-02,
        -1.01363212e-01, -2.29153130e-02, -4.96070758e-02,  4.44727466e-02,
         2.36339159e-02,  6.05530553e-02, -2.41196901e-02,  3.91690135e-02,
        -5.07083908e-03, -1.72626209e-02,  5.26820458e-02, -4.72456887e-02,
         1.85332298e-02, -2.26278715e-02,  3.78295816e-02, -8.19180743e-04,
        -8.93227831e-02,  2.79583

In [87]:
for doc in tqdm(documents):
    question = doc['Question']
    answer = doc['Answer']
    qt = question + ' ' + answer

    doc['question_vector'] = model.encode(question)
    doc['answer_vector'] = model.encode(answer)
    doc['question_answer_vector'] = model.encode(qt)

  0%|          | 0/425 [00:00<?, ?it/s]

In [88]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/425 [00:00<?, ?it/s]

In [89]:
query = "Can I get sick pay if I'm self-isolating?"
v_q = model.encode(query)

In [90]:
def elastic_search_knn(field, vector, section):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "Section": section
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["Answer", "Section", "Question", "Category", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [97]:
elastic_search_knn('question_vector', v_q, 'general claim benefits')

[{'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
  'Category': 'Temporarily unable to work',
  'Question': "Can I get sick pay if I'm self-isolating?",
  'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0',
  'Section': 'general claim benefits'},
 {'Answer': 'You need to provide a fit note from your doctor to apply for sick pay.',
  'Category': 'Temporarily unable to work',
  'Question': 'How do I apply for sick pay?',
  'id': 'ecc42084-d88e-5bf0-8070-a18552c283bb',
  'Section': 'general claim benefits'},
 {'Answer': "Statutory Sick Pay is a legal requirement for employers to pay you if you're ill.",
  'Category': 'Temporarily unable to work',
  'Question': 'What is statutory sick pay?',
  'id': 'ee6a77ec-d4ef-50f7-9762-04cc228b3a48',
  'Section': 'general claim benefits'},
 {'Answer': 'You may be eligible for certain benefits even if you’re working depending on your income and circumstances.',
  'Category': 'Manage existing benefit',
  'Question': 'Can I receive b

In [98]:
ground_truth[10]

{'question': 'Is it possible to appeal?',
 'claims_type': 'general claim benefits',
 'document': '8d000ade-6c2b-571c-aa61-5d38eb463cf8'}

In [99]:
def question_vector_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, section)

In [100]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [101]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [102]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [103]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.8637469586374696, 'mrr': 0.7441443633414435}

Compared to Minsearch: {'hit_rate': 0.8306569343065694, 'mrr': 0.6697891321978915}

Compared to Elasticsearch Text only: {'hit_rate': 0.7732360097323601, 'mrr':0.6168369829683708}

In [106]:
def answer_vector_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn('answer_vector', v_q, section)
evaluate(ground_truth, answer_vector_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.8272506082725061, 'mrr': 0.6879399837793996}

In [107]:
def question_answer_vector_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn('question_answer_vector', v_q, section)

evaluate(ground_truth, question_answer_vector_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.9304136253041363, 'mrr': 0.8123276561232763}

In [109]:
def elastic_search_knn_combined(vector, section):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "Section": section
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'answer_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_answer_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "Section": section
                    }
                }
            }
        },
        "_source": ["Answer", "Section", "Question", "Category", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [110]:
def vector_combined_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, section)

evaluate(ground_truth, vector_combined_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.9211678832116789, 'mrr': 0.8021573398215729}

question_answer_vector : {'hit_rate': 0.9304136253041363, 'mrr': 0.8123276561232763}